In [1]:
import pandas as pd 
import numpy as np
import datetime 
import os 
from isswrapper.util.helpers import read_parquet_into_dataframe
from ydata_profiling import ProfileReport
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from adtk.detector import VolatilityShiftAD, QuantileAD, PersistAD
import itertools

c:\Users\Miqasa\OneDrive\Документы\ds_projs\other\pnd_detector\.venv\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\Miqasa\OneDrive\Документы\ds_projs\other\pnd_detector\.venv\Lib\site-packages\visions\backends\shared\nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit


In [2]:
# loading data
current_path = os.getcwd()
project_path = os.path.dirname(current_path)
datasets_folder_path = os.path.join(project_path, 'datasets')
# news 
pnd_token_date_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "pnd_token_date.parquet"))
# all time series
ts_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "time_series_securities_pnd.parquet"))
news_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "final_df.parquet"))
print(ts_df.shape)
ts_df.sample(5)

(134205, 24)


,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR,currencyid
1717,TQBR,2021-04-05,ТНСэнМарЭл,MISB,13.0,89750.0,12.9,12.8,13.1,13.10000,...,12.9000,13.10000,0.0,507670.0,0.0,0.0,3,SUR,0.77,RUB
698,TQBR,2017-03-20,ТНСэнРст,RTSB,0.0,0.0,NaN,NaN,NaN,0.31000,...,0.3080,0.31000,0.0,502390.0,0.0,NaN,3,SUR,NaN,RUB
1731,TQBR,2021-04-23,АбрауДюрсо,ABRD,427.0,3088210.0,206.0,204.5,206.5,205.00000,...,205.0000,205.00000,3088210.0,3088210.0,3088210.0,0.0,3,SUR,-0.24,RUB
1152,TQBR,2019-01-03,Мостотрест,MSTT,109.0,544349.0,84.3,84.0,85.4,85.00000,...,84.5000,85.00000,544349.0,544349.0,544349.0,NaN,3,SUR,0.59,RUB
1200,TQBR,2019-03-14,ТГК-14,TGKN,0.0,0.0,NaN,NaN,NaN,0.00476,...,0.0048,0.00476,0.0,508920.0,0.0,NaN,3,SUR,-100.00,RUB


In [4]:
news_df[news_df["body"].str.contains("MSTT")]

,id,tag,title,published_at,modified_at,body
1885,55788,site,Об изменении риск-параметров на фондовом рынке...,2023-04-20 11:40:00,2023-04-20 11:42:13,<p>Решением НКО НКЦ (АО) с 24 апреля 2023 г. н...
2393,55065,site,Об изменении риск-параметров на фондовом рынке...,2023-03-14 18:21:00,2023-03-14 18:22:34,<p><span><span>С 15 марта 2023 г. решением НКО...
2782,54426,site,Об изменении риск-параметров на фондовом рынке...,2023-02-09 08:59:00,2023-02-09 09:01:34,<p><span><span>С 9 февраля 2023 г. решением НК...
12132,38271,site,Об изменении уровня листинга ценных бумаг,2021-11-30 17:31:00,2021-11-30 17:34:58,<p>В соответствии с Правилами листинга ПАО Мос...
14702,34446,site,Новые базы расчета индексов Московской биржи,2021-06-03 20:31:00,2021-06-10 14:51:12,<p>Московская Биржа по рекомендации Индексного...
17443,30593,site,"О возобновление торгов ценными бумагами ПАО ""М...",2020-10-13 19:40:00,2020-10-13 19:41:30,<p>В соответствии с Правилами листинга ПАО Мос...
17447,30589,site,О допустимых кодах расчетов по акциям обыкнове...,2020-10-13 18:06:00,2020-10-13 18:06:57,<p><span>В соответствии с Правилами проведения...
17669,30292,site,"О приостановке торгов ценными бумагами ПАО ""МО...",2020-09-23 18:50:00,2020-09-23 18:51:55,<p>В соответствии с Правилами листинга ПАО Мос...
17776,30133,site,Об изменении риск-параметров на фондовом рынке...,2020-09-15 12:13:00,2020-09-15 12:15:47,<p><span>Решением НКО НКЦ (АО) с 21 сентября 2...
18036,29780,site,Об ограничении кодов расчетов по акциям обыкно...,2020-08-18 16:27:00,2020-08-18 16:28:43,<p><span>В соответствии с Правилами проведения...


In [1]:
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt 
